In [1]:
import pandas as pd
import os
import sys
import glob
import tweepy
import numpy as np
import pandas as pd
import csv
import string
import requests
from requests_oauthlib import OAuth1
import json
import time
from google.cloud import storage

In [2]:
file_dict = {}

In [ ]:
"""
Read csv files from separate folders and create a dictionary file_dict of form {df_name: df}
"""
def read_csv(date,datelong):   
    path = '/Users/shrutigandhi/Desktop/MScA UChicago/Quarter 1/31013 Big Data/Project/CoAID-master/'+datelong+'/'
    os.chdir(path)
    extension = "csv"
    files = glob.glob('*.{}'.format(extension))
    for file in files:
        tag = "Real" if file.find("Real") > 0 else "Fake"
        source = "Claim" if file.find("News") < 0 else "News"
        tweet_reply = True if file.find("tweets_replies") > 0 else False
        tweet = True if file.find("tweets.") > 0 else False
    
        if tweet == True:
            key = source+"_"+tag+"_tweets_"+date
            df = pd.read_csv(source+tag+"COVID-19_tweets.csv")
        elif tweet_reply == True:
            key = source+"_"+tag+"_tweets_replies_"+date
            df = pd.read_csv(source+tag+"COVID-19_tweets_replies.csv")
        else:
            key = source+"_"+tag+"_"+date
            df = pd.read_csv(source+tag+"COVID-19.csv")
        file_dict[key] = df

In [ ]:
read_csv("jan5","05-01-2020")
read_csv("jan7","07-01-2020")
read_csv("jan9","09-01-2020")
read_csv("jan11","11-01-2020")

In [ ]:
# Checks
len(file_dict.keys())
file_dict.keys()
file_dict['News_Real_tweets_replies_jan5'].head()

In [ ]:
# Create tags, source, and date columns in the dfs
dates = ["jan5","jan7","jan9","jan11"]
tags = ["Fake","Real"]
sources = ["News","Claim"]

for key in file_dict.keys():
    for i in dates:
        if i in key:
            file_dict[key]["date"] = i

for key in file_dict.keys():
    for i in tags:
        if i in key:
            file_dict[key]["tag"] = i

for key in file_dict.keys():
    for i in sources:
        if i in key:
            file_dict[key]["source"] = i

In [ ]:
# append entire data
appended_data = pd.DataFrame()
for key in file_dict.keys():
    appended_data = appended_data.append([file_dict[key]])

In [ ]:
# Checks
appended_data.shape
appended_data.head()

Extracting Tweets and Uploading to GCS Bucket

In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "even-hull-328204-ce42cf63b52f.json"
client = tweepy.Client(bearer_token='AAAAAAAAAAAAAAAAAAAAAFcOVAEAAAAA448MSlftb%2Bjape1DZl0OO2LFppA%3Dp8iQlwSFlyDTdEZuyaxuGXbbgzgtblgFM63N5MIZERKx59BjBP',
                      wait_on_rate_limit=True)

In [ ]:
CONSUMER_KEY = 'RAZ2FeXVy0xA91xx2HbzNxDwv'
CONSUMER_SECRET = 'kd4kQWPbzms3dzLj1g2ngSfzsNLTJAHf3nnhvLTzSikiNA7Jjg'
OAUTH_TOKEN= '1672431301-dNVTlYEE6OlltS53tCTAGFfF47Y7j74WPcommDs'
OAUTH_TOKEN_SECRET = 'YBghh7QRPMRLI36xrct9WEcROvqLZqG9UCJrDkC268ala'

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [ ]:
"""
upload the final dataset to gcs in the specified folder
"""
def upload_to_gcp(out_df):
    storage_client = storage.Client()
    bucket_name = 'bigdata_project_hksv'
    bucket = storage_client.bucket(bucket_name)
    name = 'CoAID_cleaned_data/coaid_clean_data.csv'
    blob = bucket.blob(name)
    blob.upload_from_string(out_df.to_csv(), 'text/csv')

In [ ]:
"""
extract tweets given list of tweet_ids, and create a field in out_df to put in the tweet text
"""
def twitter_api(tweet_id_list, out_df, field, index):
    out_df.set_index(index, inplace = True)

    tweet_list = client.get_tweets(tweet_id_list)
    ids = [float(i.id) for i in tweet_list.data]
    text = [i.text for i in tweet_list.data]

    out_df.loc[ids, field] = text

    return out_df
        
#     except Exception as e:
#         print(e)
#         if '429' in str(e):
#             print(429)
#         pass

In [ ]:
def check_df(df2):
    if os.path.exists('logs.json'):
        with open('logs.json') as json_file:
            dict1 = json.load(json_file)
        df2 = df2[df2.tweet_id.isin((set(df2.tweet_id.to_list()) - set(dict1['ids'])))]
        return df2
    else:
        return df2

In [ ]:
"""
drop duplicate and missing tweet_ids/reply_ids
"""
def preprocessing(data, field):
    df1 = data[data[field].notna()]
    df1 = df1.drop_duplicates(subset=[field])
    df2 = check_df(df1)
    return df2

In [ ]:
def write_to_json(appended_data):
    appended_data.reset_index(inplace = True)
    if os.path.exists('logs.json'):
        with open('logs.json') as json_file:
            dict1 = json.load(json_file)
        for i in appended_data['tweet_id'].to_list():
            dict1['ids'].append(i)
        
        f = open('logs.json', "w")
        json.dump(dict1, f)
        f.close()
    
    else:
        dict1 = {}
        dict1['ids'] = appended_data['tweet_id'].to_list()
        f = open('logs.json', "w")
        json.dump(dict1, f)
        f.close()

In [ ]:
tweets = []
df3 = preprocessing(appended_data, "tweet_id")
df3_reply = preprocessing(appended_data, "reply_id")


tweet_id_list = list(map(int, df3['tweet_id'].tolist()))
df3.reset_index(inplace  = True, drop = True)
appended_list_new = []
for i in range(0, df3.shape[0], 100):
    tweet_ids = tweet_id_list[i: i+100]
    out_df = twitter_api(tweet_ids,df3.iloc[i:i+100],'tweet','tweet_id')
    appended_list_new.append(out_df)
appended_list_new = pd.concat(appended_list_new) # data with all tweetids and text (unique and not null)
write_to_json(appended_list_new)

In [ ]:
tweet_id_list_reply = list(map(int, df3_reply['reply_id'].tolist()))
df3_reply.reset_index(inplace  = True, drop = True)
appended_list_new_reply = []
for i in range(0, df3_reply.shape[0], 100):
    tweet_ids_reply = tweet_id_list_reply[i: i+100]
    out_df_reply = twitter_api(tweet_ids_reply,df3_reply.iloc[i:i+100],'reply','reply_id')
    appended_list_new_reply.append(out_df_reply)
appended_list_new_reply = pd.concat(appended_list_new_reply) # data with all replyids and text (unique and not null)
write_to_json(appended_list_new_reply)

In [ ]:
req_cols = appended_list_new[['tweet_id', 'tweet']] # keeping only tweet_id and tweet
appended_merge_1 = appended_data.merge(req_cols, on='tweet_id', how='left') # left_join with base data containing duplicate tweet_ids on left

In [ ]:
req_cols_reply = appended_list_new_reply[['reply_id', 'reply']] # keeping only reply_id and tweet replies
coaid_clean_data = appended_merge_1.merge(req_cols_reply, on='reply_id', how='left') # left_join with merged data created in last step on left

In [ ]:
print(coaid_clean_data[coaid_clean_data["tweet"].notna()].shape,
      coaid_clean_data[coaid_clean_data["reply"].notna()].shape)

In [ ]:
upload_to_gcp(coaid_clean_data)